# PSET 4 — Model-base Control

**Due date: November 30th, 11:59 pm**

This is the last problem set and we will cover topics from LQR control to trajectory optimization and model predictive control. You will learn how to write python code to solve a trajectory when given dynamics using numerical solver.

Problem 1 - Linear system with bounded control (25pt + 10pt bonus)
- 1.1 Prove convexity (5pt)
- 1.2 Discretization of dynamics (5pt)
- 1.3 Numerical solve trajectory optimization (10pt)
- 1.4 Resolution study (5pt)
- 1.5 KKT system (Bonus) (10pt)

Problem 2 - Use linearized dynamics to design a LQR controller (25pt + 5pt bonus)
- 2.1 Lagrangian dynamics (Bonus) (5pt)
- 2.2 State space dynamics (5pt)
- 2.3 Linearize dynamics (5pt)
- 2.4 Simulate error (5pt)
- 2.5 LQR controller (10pt)

Problem 3 - Solve obstacle avoidance problem using CRISP solver (20pt)
- 3.1 Write down the problem (10pt)
- 3.2 Solve the problem using CRISP (10pt)

Problem 4 - Model predictive control on Inverted pendulum (30pt)
- 4.1 Trajectory optimization (10pt)
- 4.2 Fail of open loop control (10pt)
- 4.3 MPC control (10pt)


## Problem 1— LQR with bounded control

In class, we studied the unconstrained LQR problem solved via Bellman’s equation. In practice, inputs and states are rarely unbounded—actuators saturate and safety limits constrain the state.

For linear systems with convex constraints on inputs and states, the seminal paper [Bemporad et al., 2002](http://cse.lab.imtlucca.it/~bemporad/publications/papers/automatica-mpqp.pdf) analyzes the structure of the optimal cost-to-go (value function) and controller. We consider a finite-horizon, discrete-time linear system with fixed matrices $A, B$, deterministic dynamics, and a quadratic cost. Inputs are **bounded**.

Given $x_0$, horizon $N$, weights $Q_k \succeq 0$ for $k=0,\dots,N$ and $R_k \succ 0$ for $k=0,\dots,N-1$, and a bound $u_{\max} > 0$, consider
$$
\begin{aligned}
J(x_0) \;=\; \min_{u_0,\dots,u_{N-1}} \;&
\sum_{k=0}^{N} x_k^\top Q_k x_k \;+\; \sum_{k=0}^{N-1} u_k^\top R_k u_k \\
\text{s.t.}\;& x_{k+1} = A x_k + B u_k,\quad k=0,\dots,N-1,\\
& u_k \in [-u_{\max},\, u_{\max}] \ \text{(componentwise)},\quad k=0,\dots,N-1.
\end{aligned}
$$




### 1.1. **Convexity.** 
Show the problem is a convex optimization problem when $x_0$ is fixed.

**Answer:**



### 1.2. Double integrator discretization

Start from the continuous-time model $\ddot q = u,\ u \in [-1,1]$. Using $x=[q,\ \dot q]^\top$ and a fixed time step $dt$, derive the discrete system $x_{k+1} = A x_k + B u_k$.

**Answer:**  



### 1.3 Numerical exploration

With $N=50$, $dt=0.1$, and your choice of $Q_k, R_k$, solve the problem for a grid of initial states $x_0$ (using cvxpy). Plot $J(x_0)$ and a representative control (e.g., $u_0(x_0)$).


In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

# =========================
# Problem setup
# =========================

####################
# TODO: Define the hyperparameters
####################
N   = 
dt  = 
umax = 

# Euler discretization for the double integrator: x = [q, qdot]
A = 
B = 

# Cost weights (time-invariant)
Q  = 
R  = 

n, m = A.shape[0], B.shape[1]

# =========================
# Build one QP (with Parameter x0) that we can reuse for all grid points
# =========================

####################
# TODO: Define problem using cvxpy
####################

x0_param = cp.Parameter(n)  # initial condition parameter

prob = cp.Problem(...)

# =========================
# Grid of initial states
# =========================

####################
# TODO: Plot here
####################




### 1.4 Resolution study

Increase $N$, decrease $dt$, and plot $J,u$ on a denser grid of initial states. What can you observe from the plots? Comment on solver runtime.

**Answer:**




### 1.5 KKT conditions (Bonus)
Write the KKT optimality conditions for the problem.

**Answer:**  


## Problem 2 — Cart–Pole system (linearization + LQR)

We consider a cart of mass $m_c$ moving on a horizontal track with a pendulum (mass $m_p$, length $l$) hinged to the cart. The input $f$ is a horizontal force applied to the cart. We assume no friction and model the pendulum as a point mass at its tip. See the [Video](https://www.youtube.com/watch?v=Bzq96V1yN5k).

<figure style="text-align:center;">
  <img src="https://raw.githubusercontent.com/ComputationalRobotics/2025-ES-AM-158-PSET/main/PSET4/cartpole.png" width="360" alt="Inscribed polygon">
  <figcaption style="color:#6a737d; font-style:italic;">
    Figure 1. Cartpole.
  </figcaption>
</figure>

With this setup, we parameterize the system by two scalars: $x$ denotes the cart position, and $\theta$ is the angle between the pole and the stable equilibrium. Our goal is to study the cart–pole dynamics under the horizontal control force $f$.




### 2.1 Lagrangian derivation (Bonus)
For those of you who have background in rigid-body dynamics, this is an opportunity for you to apply your knowledge. However, feel free to skip this subproblem and it won’t affect the rest of the exercise. Derive the equations of motion:

$$
\begin{aligned}
(m_c + m_p)\,\ddot{x} + m_p\,l\,\ddot{\theta}\cos\theta - m_p\,l\,\dot{\theta}^2\sin\theta &= f, \\ 
m_p\,l\,\ddot{x}\cos\theta + m_p\,l^2\,\ddot{\theta} + m_p\,g\,l\,\sin\theta &= 0.
\end{aligned}
$$
(Hints: compute the Lagrangian of the system and the corresponding Lagrangian equations. Analyzing the two objects separately also works.)

**Answer:**


### 2.2 State–space form.
Let $x_1=x$, $x_2=\theta$, $x_3=\dot x$, $x_4=\dot\theta$, and input $u=f$. Translate the equations in 2.1 into the basic state-space dynamics form $\dot{\mathbf x}=F(\mathbf x,u)$.

**Answer:**  


### 2.3 Linearize about the upright equilibrium. 
Linearize around the unstable equilibrium point $x^*=0$, $\dot x^*=0$, $\theta^*=\pi$, $\dot\theta^*=0$, $u^*=0$  (i.e., the pole is in the upright position and the cart stay at zero.) to obtain $\dot{\Delta x}=A\,\Delta x+B\,\Delta u$ with $\Delta x=x-x^*$, $\Delta u=u-u^*$. Write down what is $A$ and $B$.

**Answer:**  


### 2.4 Linearization Error

Define the linearization error
$e(\mathbf{x}, \mathbf{u}) = \|\,F(\mathbf{x}, \mathbf{u}) - (A\,\Delta\mathbf{x} + B\,\Delta\mathbf{u})\,\|^2$ (i.e., the error between the true nonlinear system and the linearized system).
Simulate both the nonlinear system and its linearization from several initial conditions, and plot how this error evolves over time.

**Simulation**

A minimal forward–Euler loop is:

```
for i in range(100):
    x = x + dt * F(x,u)
```

But here we use the [Runge-Kutta](https://en.wikipedia.org/wiki/Runge–Kutta_methods) format.
```
for i in range(100):
    x = RK4(F(as a handle), x, u, dt)
```

See the provided `rk4_step(f, x, u, dt)` function for details.

**TODO:** Finish the code block.

In [ ]:
# Cart-pole: simulate nonlinear vs. linearized model and plot linearization error.
# Requirements: numpy, matplotlib

import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# Physical parameters (edit if needed)
# -----------------------------
m_c = 1.0   # cart mass [kg]
m_p = 0.1   # pole mass [kg]
l   = 0.5   # pole length [m]
g   = 9.81  # gravity [m/s^2]

# Upright equilibrium (x*, theta*, xdot*, thetadot*, u*)
x_star = np.array([0.0, np.pi, 0.0, 0.0])
u_star = np.array([0.0])

# -----------------------------
# Dynamics
# -----------------------------
def F(x, u):
    """
    Nonlinear cart-pole dynamics: x=[x, theta, xdot, thetadot]
    returns [xdot, thetadot, xddot, thetaddot]
    """
    ####################
    # TODO: Define the Nonlinear Dynamics here
    ####################
    


def F_lin_state(x, u):
    """
    Linearized dynamics about (x_star, u_star):
        xdot ≈ A (x - x_star) + B (u - u_star)
    using the provided A and B matrices.
    """
    ####################
    # TODO: Define the linear Dynamics here
    ####################
    


# -----------------------------
# Integrator (RK4)
# -----------------------------
def rk4_step(f, x, u, dt):
    k1 = f(x, u)
    k2 = f(x + 0.5*dt*k1, u)
    k3 = f(x + 0.5*dt*k2, u)
    k4 = f(x + dt*k3, u)
    return x + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)


# -----------------------------
# Input profiles (edit or add your own)
# -----------------------------
def make_u_fun(kind="zero", amp=5.0, t0=0.5, width=0.2, freq=1.0):
    """
    kind: "zero" | "step" | "pulse" | "sine"
    amp:  amplitude (N)
    t0:   step/pulse start (s)
    width:pulse width (s)
    freq: sine frequency (Hz)
    """
    if kind == "zero":
        return lambda t, x: 0.0
    if kind == "step":
        return lambda t, x: (amp if t >= t0 else 0.0)
    if kind == "pulse":
        return lambda t, x: (amp if (t0 <= t <= t0+width) else 0.0)
    if kind == "sine":
        return lambda t, x: amp*np.sin(2*np.pi*freq*t)
    raise ValueError("unknown kind")


# -----------------------------
# Simulation
# -----------------------------
def simulate_cartpole_error(x0, T=5.0, dt=0.01, u_kind="zero", **u_kwargs):
    """
    Simulate nonlinear and linearized models from initial state x0.
    Returns dict with time, states, and error e(t).
    """
    u_fun = make_u_fun(u_kind, **u_kwargs)

    ts = np.arange(0.0, T + dt, dt)
    X_nl = np.zeros((len(ts), 4))
    X_li = np.zeros((len(ts), 4))
    err  = np.zeros(len(ts))

    x_nl = x0.copy()
    x_li = x0.copy()

    for k, t in enumerate(ts):
        u = np.array([u_fun(t, x_nl)])

        # store
        X_nl[k] = x_nl
        X_li[k] = x_li

        # linearization error at the *nonlinear* state (definition in the exercise)
        e_vec = F(x_nl, u) - F_lin_state(x_nl, u)
        err[k] = float(np.dot(e_vec, e_vec))  # squared 2-norm

        # advance one step
        x_nl = rk4_step(F, x_nl, u, dt)
        x_li = rk4_step(F_lin_state, x_li, u, dt)

    return {"t": ts, "X_nl": X_nl, "X_lin": X_li, "e": err}


# -----------------------------
# Plotting
# -----------------------------
def plot_results(sim):
    ####################
    # TODO: plot result here
    ####################

x0 = np.array([0.0, np.pi + np.deg2rad(2.0), 0.0, 0.0])  # 10° from upright

sim = simulate_cartpole_error(
    x0,
    T=2.0,
    dt=0.01,
    u_kind="pulse",   # "zero", "step", "pulse", or "sine"
    amp=5.0,
    t0=0.5,
    width=0.2,
    freq=1.0,
)
plot_results(sim)


### 2.5 Discrete-time model and LQR design.
Convert the linear model in 2.3 to discrete time with step $dt$, then design a discrete LQR to stabilize the upright equilibrium. Does it work for all initial conditions?

**TODO:** Get the linear feedback control gain $K$ and simulate. Finish the code block.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.linalg import solve_discrete_are


# -----------------------------
# Parameters
# -----------------------------
m_c = 1.0   # cart mass [kg]
m_p = 0.1   # pole mass [kg]
l   = 0.5   # pole length [m]
g   = 9.81  # gravity [m/s^2]

dt = 0.02   # discretization step for LQR and simulation [s]
T  = 6.0    # total simulation time [s]
N  = int(T / dt)

# LQR weights (tune as you like)
Q = np.diag([1.0, 50.0, 0.2, 1.0])  # penalize angle deviation strongly
R = np.array([[0.1]])               # control effort penalty

u_max = 10.0   # actuator saturation (set to None to disable)

# -----------------------------
# Nonlinear dynamics + integrator
# -----------------------------
def F_nonlinear(x, u, m_c=1.0, m_p=0.1, l=0.5, g=9.81):
    ####################
    # TODO: Finish the nonlinear dynamics here
    ####################
    # x = [x, theta, xdot, thetadot]
    
def rk4_step(f, x, u, dt):
    k1 = f(x, u)
    k2 = f(x + 0.5*dt*k1, u)
    k3 = f(x + 0.5*dt*k2, u)
    k4 = f(x + dt*k3, u)
    return x + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)

# -----------------------------
# Build controller K, then simulate NONLINEAR plant
# -----------------------------
####################
# TODO: Build the K here
####################
K      = 

def simulate_nonlinear_from(x0, K, dt, N, u_max=None):
    """
    x0 is full state: [x, theta, xdot, thetadot].
    Control law: u = -K @ [Δx, Δθ, Δxdot, Δthetadot], where Δθ = theta - pi.
    """
    X = np.zeros((N+1, 4))
    U = np.zeros((N, 1))
    X[0] = x0
    for k in range(N):
        dx = np.array([
            X[k, 0],            # Δx (cart position about 0)
            X[k, 1] - np.pi,    # Δθ about upright
            X[k, 2],            # Δxdot
            X[k, 3],            # Δthetadot
        ])
        u = -K @ dx
        if u_max is not None:
            u = np.clip(u, -u_max, u_max)
        U[k, 0] = u
        X[k+1] = rk4_step(lambda x, uu: F_nonlinear(x, uu, m_c, m_p, l, g), X[k], u, dt)
    return X, U

x0 = np.array([0.0, np.pi + np.deg2rad(30.0), 0.0, 0.0])

X, U = simulate_nonlinear_from(x0, K, dt, N, u_max=u_max)

# -----------------------------
# Plot
# -----------------------------
t  = np.linspace(0.0, T, N+1)
tk = np.linspace(0.0, T, N)

plt.figure(figsize=(9, 4))
plt.plot(t, X[:,1] - np.pi)
plt.xlabel("time (s)")
plt.ylabel("Δθ = θ - π (rad)")
plt.title("Nonlinear cart–pole under LQR (angle deviation)")
plt.grid(True, alpha=0.3)
plt.show()

plt.figure(figsize=(9, 4))
plt.plot(t, X[:,0])
plt.xlabel("time (s)")
plt.ylabel("x (m)")
plt.title("Cart position (nonlinear plant)")
plt.grid(True, alpha=0.3)
plt.show()

plt.figure(figsize=(9, 4))
plt.step(tk, U[:,0], where='post')
plt.xlabel("time (s)")
plt.ylabel("u (N)")
plt.title("Control input (with saturation)" if u_max is not None else "Control input")
plt.grid(True, alpha=0.3)
plt.show()

print("Final angle deviation (deg):", np.rad2deg(X[-1,1] - np.pi))
print("Did it stabilize? (|Δθ| < 2° at end):", abs(np.rad2deg(X[-1,1] - np.pi)) < 2.0)


## Problem 3 — Obstacle Avoidance with the CRISP Solver

Adapt Example 4.4 from the lecture notes to implement a trajectory optimization in Python using the CRISP solver. See the paper for details: [CRISP](https://computationalrobotics.seas.harvard.edu/CRISP/). Run the code locally, or copy `crisp.py` into your Google Colab folder.

CRISP need QP solver, for example I use PIQP here. Install the solver using `pip install 'qpsolvers[piqp]'`. You may change to other solvers by looking into CRISP's source code.

We consider a unicycle robot that must travel from a start pose **A** to a goal pose **B** while avoiding circular (disk) obstacles.

#### System Model
We use standard unicycle (Dubins-like) kinematics in continuous time:
$$
\dot{x}(t)
= \begin{bmatrix}\dot p_x\\ \dot p_y\\ \dot \theta\end{bmatrix}
= \begin{bmatrix} v(t)\cos\theta(t)\\ v(t)\sin\theta(t)\\ \omega(t) \end{bmatrix},\quad
x=\begin{bmatrix}p_x& p_y& \theta\end{bmatrix}^\top,\;
u=\begin{bmatrix}v&\omega\end{bmatrix}^\top.
$$

Discretize on a uniform grid $t_k=kh$ for $k=0,\ldots,N$ with step $h>0$ using forward Euler:
$$
x_{k+1} = f_h(x_k,u_k)
:= \begin{bmatrix}
p_{x,k} + h\, v_k \cos\theta_k\\
p_{y,k} + h\, v_k \sin\theta_k\\
\theta_k + h\, \omega_k
\end{bmatrix}.
$$

#### Decision Variables
Optimize over $\{x_k\}_{k=0}^{N}$ and $\{u_k\}_{k=0}^{N-1}$, collected as
$$
z=\big[ x_0^\top,\; u_0^\top,\; x_1^\top,\; u_1^\top,\; \ldots,\; x_{N-1}^\top,\; u_{N-1}^\top,\; x_N^\top \big]^\top
\in \mathbb{R}^{(3+2)N+3}.
$$

#### Constraints

**(i) Initial condition.**
$$
x_0 = A \in \mathbb{R}^3.
$$

**(ii) Dynamics (equalities), for $k=0,\ldots,N-1$.**
$$
x_{k+1} - f_h(x_k,u_k) = 0.
$$

**(iii) Obstacle avoidance (inequalities).**  
Let $\mathcal{O}=\{(c_j,r_j)\}_{j=1}^{n_{\text{obs}}}$ with centers $c_j=\begin{bmatrix}c_{x,j}&c_{y,j}\end{bmatrix}^\top$ and radii $r_j>0$. Enforce a safety margin $\delta\ge 0$ at each knot:
$$
(p_{x,k}-c_{x,j})^2 + (p_{y,k}-c_{y,j})^2 \;\ge\; (r_j+\delta)^2,
\qquad \forall k=0,\ldots,N,\ \forall j.
$$

**(iv) Box limits (controls, optionally states).**
$$
v_{\min}\le v_k \le v_{\max},\qquad
\omega_{\min}\le \omega_k \le \omega_{\max},\qquad
k=0,\ldots,N-1.
$$


### 4.1 Write down your optimization problem 

According to the dynamics and the constraints, design the loss function yourself and write down the problem formulation as a mathematical optimization problem.

**Answer:**

### 4.2 Program your problem and solve it using CRISP

Crisp's interface allow you to solve the problem 

$$
\begin{align}
& \min_x f(x) \\
s.t. & \quad h(x) = 0\\
& \quad g(x) \geq 0\\
\end{align}
$$

When given the cost function $f$, its gradient and hessian $\nabla f, \nabla^2 f$, the equality and inequality constraint $h(x), g(x)$, and the jacobian of the constraint $\frac{\partial h}{\partial x}$ and $\frac{\partial g}{\partial x}$.

The code below provides a simple example:


In [3]:
from crisp import SQPSolver
import numpy as np

def cost(x):
    return x.dot(x)

def grad_cost(x):
    return 2*x

def hess_cost(x):
    return 2*np.eye(x.size)

def eq_cons(x):
    return np.array([ x[0] + x[1] - 1 , 
                        x[2] - x[3] + 1])

def jac_eq(x):
    return np.vstack([
        np.array([1,1,0,0,0]),
        np.array([0,0,1,-1,0])
    ])

def ineq_cons(x):
    
    return np.array([ x[2], x[3] ])

def jac_in(x):
    return np.vstack([
        np.array([0,0,1,0,0]),
        np.array([0,0,0,1,0])
    ])


x0 = np.zeros(5)

solver = SQPSolver(cost, grad_cost, hess_cost,
                    eq_cons, jac_eq,
                    ineq_cons, jac_in,
                    x0,
                    mu_eq_init=np.ones(2),
                    mu_in_init=np.ones(2),
                    max_iter=100)
x_opt, history = solver.solve()
print("Optimal x:", x_opt)

TR    predicted = 0.750000 , actual = 0.750000 , cost = 0.000000 , merit = 2.000000 , eq_vio = 1.000000 , in_vio = 0.000000 , QP_time = 0.000258s
REJ   predicted = 0.000000 , actual = 0.000000 , cost = 0.749971 , merit = 1.250000 , eq_vio = 0.499978 , in_vio = 0.000022 , QP_time = 0.000149s
Increase lower than tolerance at iter 1
TR+   predicted = 2.187760 , actual = 2.187760 , cost = 0.749971 , merit = 5.750260 , eq_vio = 0.499978 , in_vio = 0.000022 , QP_time = 0.000144s
TR    predicted = 2.062500 , actual = 2.062500 , cost = 1.062500 , merit = 3.562500 , eq_vio = 0.249949 , in_vio = 0.000051 , QP_time = 0.000134s
REJ   predicted = 0.000000 , actual = 0.000000 , cost = 1.500000 , merit = 1.500000 , eq_vio = 0.000000 , in_vio = 0.000000 , QP_time = 0.000146s
Solved at iter 4
Optimal x: [ 5.00000000e-01  5.00000000e-01  1.49686849e-10  1.00000000e+00
 -1.67234840e-24]


Then your task is to implement the obstacle avoidance problem using CRISP. The code structure is given, but you need to fill in objective, constraints and its derivative.

**TODO:** Finish `TODO` functions.

In [ ]:
from __future__ import annotations

import math
from dataclasses import dataclass
from typing import Tuple

import numpy as np
from scipy import sparse as sp
import matplotlib.pyplot as plt

from crisp import SQPSolver  


# -------------------- Problem parameters --------------------
@dataclass
class Params:
    N: int = 60
    h: float = 0.1
    A: np.ndarray = np.array([0.0, 0.0, 0.0])
    B: np.ndarray = np.array([6.0, 5.0, 0.0])
    obstacles: np.ndarray = np.array([
        [2.0, 1.5, 0.8],
        [3.5, 3.5, 0.9],
        [5.2, 3.8, 0.7],
        [2.2, 3.2, 0.3],
        [4.5, 1.0, 1.0],
        [5.0, 2.5, 0.05],
    ])
    safety_margin: float = 0.25

    v_min: float = -2.0
    v_max: float = 2.0
    w_min: float = -2.5
    w_max: float = 2.5

    w_goal_pos: float = 20.0
    w_goal_theta: float = 20.0
    w_u: float = 0.05
    w_du: float = 0.2

    n_x: int = 3
    n_u: int = 2

    def __post_init__(self):
        self.Z_DIM = (self.n_x + self.n_u) * self.N + self.n_x


# -------------------- Index helpers --------------------
def idx_x(k: int, p: Params) -> slice:
    base = (p.n_x + p.n_u) * k
    return slice(base, base + p.n_x)


def idx_u(k: int, p: Params) -> slice:
    base = (p.n_x + p.n_u) * k + p.n_x
    return slice(base, base + p.n_u)


# -------------------- Pack / Unpack --------------------
def pack(X: np.ndarray, U: np.ndarray, p: Params) -> np.ndarray:
    z = np.zeros(p.Z_DIM, dtype=float)
    for kk in range(p.N):
        z[idx_x(kk, p)] = X[kk, :]
        z[idx_u(kk, p)] = U[kk, :]
    z[idx_x(p.N, p)] = X[p.N, :]
    return z


def unpack(z: np.ndarray, p: Params) -> Tuple[np.ndarray, np.ndarray]:
    X = np.zeros((p.N + 1, p.n_x), dtype=float)
    U = np.zeros((p.N, p.n_u), dtype=float)
    for kk in range(p.N):
        X[kk, :] = z[idx_x(kk, p)]
        U[kk, :] = z[idx_u(kk, p)]
    X[p.N, :] = z[idx_x(p.N, p)]
    return X, U


# -------------------- Dynamics --------------------
def f_disc(xk: np.ndarray, uk: np.ndarray, h: float) -> np.ndarray:
    px, py, th = xk
    v, w = uk
    return np.array([
        px + h * v * math.cos(th),
        py + h * v * math.sin(th),
        th + h * w,
    ])

# -------------------- Objective & derivatives --------------------
def objective(z: np.ndarray, p: Params) -> float:
    ####################
    # TODO: Finish this
    ####################
    


def grad_cost(z: np.ndarray, p: Params) -> np.ndarray:
    ####################
    # TODO: Finish this
    ####################
    


def hess_cost(z: np.ndarray, p: Params):
    ####################
    # TODO: Finish this
    ####################
    


# -------------------- Constraints --------------------
def eq_cons(z: np.ndarray, p: Params) -> np.ndarray:
    ####################
    # TODO: Finish this
    ####################
    


def jac_eq(z: np.ndarray, p: Params):
    ####################
    # TODO: Finish this
    ####################



def ineq_cons(z: np.ndarray, p: Params) -> np.ndarray:
    ####################
    # TODO: Finish this
    ####################
    """Obstacle clearance + control box constraints, all as g(z) >= 0."""
    


def jac_in(z: np.ndarray, p: Params):
    ####################
    # TODO: Finish this
    ####################
    """Sparse Jacobian of ineq_cons matching the order in ineq_cons."""



# -------------------- Plotting --------------------
def plot_results(p: Params, X: np.ndarray, U: np.ndarray) -> None:
    blue = (0.0, 0.4470, 0.7410)

    fig = plt.figure(figsize=(8, 6.5), dpi=120)
    ax = fig.add_subplot(111)
    ax.set_title("Unicycle TO (Custom SQP + Control Bounds)")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.grid(True)
    ax.set_aspect("equal", adjustable="box")

    ang = np.linspace(0, 2 * math.pi, 200)
    for j in range(p.obstacles.shape[0]):
        cx, cy, r = p.obstacles[j]
        r_eff = r + p.safety_margin
        ox = cx + r_eff * np.cos(ang)
        oy = cy + r_eff * np.sin(ang)
        ax.plot(ox, oy, "r-", linewidth=1.2)
        ax.fill(ox, oy, color=(1, 0, 0), alpha=0.12, edgecolor="none")

    ax.plot(p.A[0], p.A[1], "go", label="start")
    ax.plot(p.B[0], p.B[1], "b*", markersize=10, label="goal")

    ax.plot(X[:, 0], X[:, 1], color=blue, linewidth=2.0, label="optimized")

    skip = max(1, p.N // 20)
    for k in range(0, p.N + 1, skip):
        px, py, th = X[k]
        ax.quiver(px, py, 0.4 * math.cos(th), 0.4 * math.sin(th),
                  angles='xy', scale_units='xy', scale=1, color=blue, width=0.004)

    ax.legend(loc="best")

    t = np.arange(p.N) * p.h
    fig2 = plt.figure(figsize=(8.5, 5.0), dpi=120)

    ax1 = fig2.add_subplot(211)
    ax1.plot(t, U[:, 0], color=blue, label="v*")
    ax1.axhline(p.v_min, linestyle=":", color="k")
    ax1.axhline(p.v_max, linestyle=":", color="k")
    ax1.set_ylabel("v [m/s]")
    ax1.grid(True)
    ax1.legend()

    ax2 = fig2.add_subplot(212)
    ax2.plot(t, U[:, 1], color=blue, label="omega*")
    ax2.axhline(p.w_min, linestyle=":", color="k")
    ax2.axhline(p.w_max, linestyle=":", color="k")
    ax2.set_xlabel("time [s]")
    ax2.set_ylabel("omega [rad/s]")
    ax2.grid(True)
    ax2.legend()

    plt.tight_layout()
    plt.show()

p = Params()
X0, U0 = np.zeros((p.N + 1, p.n_x)), np.zeros((p.N, p.n_u))
z0 = pack(X0, U0, p)

cost_fn = lambda z: objective(z, p)
grad_fn = lambda z: grad_cost(z, p)
hess_fn = lambda z: hess_cost(z, p)
eq_fn   = lambda z: eq_cons(z, p)
jeq_fn  = lambda z: jac_eq(z, p)
in_fn   = lambda z: ineq_cons(z, p)
jin_fn  = lambda z: jac_in(z, p)

m_eq = 3 + 3 * p.N
m_in = (p.N + 1) * p.obstacles.shape[0] + 4 * p.N  # add 4 bounds per stage
mu_eq_init = 1.0 * np.ones(m_eq)
mu_in_init = 1.0 * np.ones(m_in)

solver = SQPSolver(cost_fn, grad_fn, hess_fn,
                    eq_fn, jeq_fn,
                    in_fn, jin_fn,
                    x0=z0,
                    mu_eq_init=mu_eq_init,
                    mu_in_init=mu_in_init,
                    delta_init=0.5,
                    delta_max=2.0,
                    tol=1e-6,
                    max_iter=2000)

z_star, hist = solver.solve()

X_star, U_star = unpack(z_star, p)
print("\nTerminal state [px py theta]^T = [" +
        f"{X_star[-1,0]:.3f} {X_star[-1,1]:.3f} {X_star[-1,2]:.3f}]")

plot_results(p, X_star, U_star)

## Problem 4: Model Predictive Control for Pendulum Swing-Up

A simple pendulum consists of a point mass $m$ attached to a massless rigid rod of length $\ell$ that pivots freely under gravity $g$. We measure the angle $\theta$ from the **upright** position ($\theta=0$), with angular velocity $\omega=\dot{\theta}$, so the state is $x=[\theta,\omega]^\top$ and the control input is a torque $\tau$ applied at the pivot. The classic **swing-up** problem starts near the downward equilibrium ($\theta\approx\pi,\ \omega\approx0$) and seeks a torque sequence that drives the pendulum to the upright ($\theta=0,\ \omega=0$) efficiently while respecting actuator limits.

<figure style="text-align:center;">
  <img src="https://raw.githubusercontent.com/ComputationalRobotics/2025-ES-AM-158-PSET/main/PSET4/invertedpendulum.png" width="360" alt="Pendulum schematic">
  <figcaption style="color:#6a737d; font-style:italic;">
    Figure 2. Pendulum.
  </figcaption>
</figure>

**State / input:** $x=\begin{bmatrix}\theta\\\omega\end{bmatrix}$, $u=\tau$ (torque)  

**Parameters:** mass $m$, length $\ell$, gravity $g$.

**Bounds:** $\tau(t)\in[\tau_{\min},\tau_{\max}]$.

**Dynamics**
$$
\dot\theta=\omega,\qquad
\dot\omega=-\frac{g}{\ell}\sin\theta+\frac{1}{m\ell^2}\,\tau .
$$

**Initial condition**
$$
x(0)=\begin{bmatrix}\pi\\0\end{bmatrix}\quad(\text{downward at rest}).
$$

**Objective**
$$
\min_{\tau(\cdot)} \int_0^{T}
\Big[
q_a\big(\sin^2\theta+(\cos\theta-1)^2\big)
+ q_\omega\,\omega^2
+ q_r \,\tau^2
\Big]\,dt
$$
subject to the dynamics, initial condition, and torque bounds.

Below is a simulation environment to test your controller.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def simulate_pendulum(u_fun,
                      x0=(0.01, 0.0), T=10.0, dt=0.01,
                      m=1.0, l=1.0, g=9.81,
                      tau_limits=None,
                      save_gif=False, gif_path="pendulum.gif", fps=None):
    """
    Simulate θ (upright=0) with torque policy u_fun(t, x) -> scalar.
    Returns (t, X, U). If save_gif, writes an animated GIF.
    """
    N = int(T/dt); t = np.linspace(0, T, N+1)
    X = np.zeros((N+1, 2)); X[0] = np.array(x0, float)
    U = np.zeros(N)

    def f(x, tt):
        th, w = x
        u = float(u_fun(tt, x))
        if tau_limits is not None: u = float(np.clip(u, tau_limits[0], tau_limits[1]))
        return np.array([w, (g/l)*np.sin(th) + u/(m*l*l)])

    def rk4(x, tt, h):
        k1 = f(x, tt)
        k2 = f(x + 0.5*h*k1, tt + 0.5*h)
        k3 = f(x + 0.5*h*k2, tt + 0.5*h)
        k4 = f(x + h*k3,     tt + h)
        return x + (h/6.0)*(k1 + 2*k2 + 2*k3 + k4)

    for k in range(N):
        th, w = X[k]
        u = float(u_fun(t[k], X[k]))
        if tau_limits is not None: u = float(np.clip(u, tau_limits[0], tau_limits[1]))
        U[k] = u
        X[k+1] = rk4(X[k], t[k], dt)

    if save_gif:
        xx_full = l*np.sin(X[:,0])
        yy_full = l*np.cos(X[:,0])

        stride = 10
        idx = np.arange(0, N+1, stride)
        xx, yy = xx_full[idx], yy_full[idx]
        fig, ax = plt.subplots(figsize=(5,5))
        ax.set_aspect("equal", adjustable="box")
        ax.set_xlim(-1.15*l, 1.15*l); ax.set_ylim(-1.15*l, 1.15*l); ax.axis("off")
        ax.plot(0, 0, "o", ms=6, color="black")
        rod, = ax.plot([0, xx[0]], [0, yy[0]], lw=3, alpha=0.9)
        bob = plt.Circle((xx[0], yy[0]), 0.06*l, ec="black", lw=1.2, fc="#87CEFA")
        ax.add_patch(bob)
        def update(i):
            rod.set_data([0, xx[i]], [0, yy[i]])
            bob.center = (xx[i], yy[i])
            return rod, bob
        out_fps = max(1, int(1/(dt*stride))) if fps is None else fps
        ani = FuncAnimation(fig, update, frames=len(idx), interval=1000*dt*stride, blit=True)
        ani.save(gif_path, writer="pillow", fps=out_fps)
        plt.close(fig)

    return t, X, U


t, X, U = simulate_pendulum(lambda tt, x: 0.0, save_gif=True)



### 4.1 Trajectory optimization
With $N=50$ and $dt=0.1$, solve the trajectory optimization problem of swinging up the pendulum from the bottom right position: parameterize controls, discretize dynamics on each interval, and enforce control bound constraints. Plot your optimal trajectory and control signal.

This problem is similar to Problem 3.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# Problem setup (pendulum)
# -----------------------------
N = 50
dt = 0.1
T = N * dt
m, l, g = 1.0, 1.0, 9.81
x0 = np.array([np.pi, 0.0])  # [theta (upright=0), omega]

# cost weights
q_a, q_omega, q_r = 20.0, 2.0, 1e-2
u_min, u_max = -2.0, 2.0

def f_pend(x, u):
    th, w = x
    return np.array([w, (g/l)*np.sin(th) + u/(m*l*l)], dtype=float)

# -----------------------------
# Decision variables: z = [th_0..th_N, w_0..w_N, u_0..u_{N-1}]
# -----------------------------
def unpack(z):
    th = z[0:N+1]
    w  = z[N+1:2*(N+1)]
    u  = z[2*(N+1):]
    return th, w, u

def pack(th, w, u):
    return np.concatenate([th, w, u], axis=0)

# -----------------------------
# Objective (pendulum swing-up)
# J ≈ ∑ dt * [ q_a( sin^2 th + (cos th - 1)^2 ) + q_omega w^2 + q_r u^2 ]  (+ terminal state term)
# -----------------------------
def objective(z):
    ####################
    # TODO: Build objective here
    ####################

# -----------------------------
# Multiple-shooting equality constraints (Euler)
# -----------------------------
def dynamics_constraints(z):
    ####################
    # TODO: Build the dynamic constraints here
    ####################

# -----------------------------
# Bounds
# -----------------------------
####################
# TODO: Finish Bounds
####################

# -----------------------------
# Feasible initial guess (u=0 forward Euler)
# -----------------------------
####################
# TODO: Finish Initial bound
####################


# -----------------------------
# Solve
# -----------------------------
res = 

print("\nSuccess:", res.success, "| Status:", res.status)
print("Message:", res.message)
print("Final objective:", res.fun)

# -----------------------------
# Extract & verify by RK4 sim with u_k (piecewise-constant)
# -----------------------------
th_sol, w_sol, u_sol = unpack(res.x)
t_nodes = np.linspace(0.0, T, N+1)
t_ctrl  = np.linspace(0.0, T - dt, N)


# -----------------------------
# Plots
# -----------------------------
fig, axes = plt.subplots(3, 1, figsize=(10, 9), sharex=True)

axes[0].plot(t_nodes, th_sol, label="theta (OCP, Euler nodes)")
axes[0].set_ylabel("theta [rad]"); axes[0].legend(); axes[0].grid(alpha=0.3)

axes[1].plot(t_nodes, w_sol, label="omega (OCP, Euler nodes)")
axes[1].set_ylabel("omega [rad/s]"); axes[1].legend(); axes[1].grid(alpha=0.3)

axes[2].step(t_ctrl, u_sol, where='post', label="tau (control)")
axes[2].axhline(u_min, ls='--', c='k', lw=0.8); axes[2].axhline(u_max, ls='--', c='k', lw=0.8)
axes[2].set_xlabel("time [s]"); axes[2].set_ylabel("tau [N·m]"); axes[2].legend(); axes[2].grid(alpha=0.3)

plt.tight_layout(); plt.show()


### 4.2 Failure of open loop control

Now we solved out a trajectory with a given initial state. However, does this imply that if we simply run the planned controls on a real system, i.e., doing open-loop control, the trajectory will behave as planned? Unfortunately the answer is No, trajectory optimization only produces an approximate solution to the optimal control problem (OCP), regardless of what transcription method is used. So the error may cumulate along time horizon.

Rollout the control to see the fail of swing up. 

**TODO:** Finish the code and comment on what you see.

In [ ]:
def u_fun(t, x):
    k = int(t // dt)
    if k < 0: k = 0
    if k >= len(u_sol): k = len(u_sol) - 1
    return float(u_sol[k])  # open-loop (ignores x)

# --- “real-world” simulation (can be finer dt than planning; choose dt_sim) ---
dt_sim = 0.01  # finer integration for verification
t_sim, X_sim, U_sim = simulate_pendulum(
    u_fun,
    x0=x0, T=T,
    tau_limits=(u_min, u_max),
    save_gif=True  # set True to export an animation
)

###################
# TODO: Plot here
###################

### 4.3 Model Predictive Control

Now we will use model predictive control to get a feedback controller. 
Basically at every time step, we solve a trajectory optimization problem to obtain an open-loop state-control trajectory, but we only execute the first control input. 
Executing the first control input will bring the system to a new state, and we resolve the trajectory optimization problem with this new initial state.
When solving the new TO problem, you can reuse the solution from previous solve as the initial guess.

**TODO:** Finish the code and plot the final trajectory

In [ ]:
##############################
# TODO: reuse your code in 4.1 to finish this
##############################